# MLP Classification with TREC Dataset
<hr>

We will build a text classification model using Multi Layer Perceptron on the TREC Dataset. Since there is no standard train/test split for this dataset, we will use 10-Fold Cross Validation (CV). 

## Load the library

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.model_selection import KFold
from scipy.io import savemat

%config IPCompleter.greedy=True
%config IPCompleter.use_jedi=False



Bad key text.latex.unicode in file C:\Users\Diardano Raihan\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle, line 112 ('text.latex.unicode : False # use "ucs" and "inputenc" LaTeX packages for handling')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.3.2/matplotlibrc.template
or from the matplotlib source distribution

Bad key savefig.frameon in file C:\Users\Diardano Raihan\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle, line 423 ('savefig.frameon : True')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.3.2/matplotlibrc.template
or from the matplotlib source distribution

Bad key pgf.debug in file C:\Users\Diardano Raihan\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle, line 444 ('pgf.debug           : False')
You probably need to get an updated matplotlibrc file from
h

In [2]:
tf.config.list_physical_devices('GPU') 

[]

## Load the Dataset

In [3]:
corpus = pd.read_pickle('../../../0_data/TREC/TREC.pkl')
corpus.label = corpus.label.astype(int)
print(corpus.shape)
corpus

(5952, 3)


,sentence,label,split
0,how did serfdom develop in and then leave russ...,0,train
1,what films featured the character popeye doyle ?,1,train
2,how can i find a list of celebrities ' real na...,0,train
3,what fowl grabs the spotlight after the chines...,1,train
4,what is the full form of .com ?,2,train
...,...,...,...
5947,who was the 22nd president of the us ?,3,test
5948,what is the money they use in zambia ?,1,test
5949,how many feet in a mile ?,5,test
5950,what is the birthstone of october ?,1,test


In [4]:
corpus.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5952 entries, 0 to 5951
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   sentence  5952 non-null   object
 1   label     5952 non-null   int32 
 2   split     5952 non-null   object
dtypes: int32(1), object(2)
memory usage: 116.4+ KB


In [5]:
corpus.groupby( by='label').count()

,sentence,split
label,,
0,1300,1300
1,1344,1344
2,95,95
3,1288,1288
4,916,916
5,1009,1009


In [6]:
# Separate the sentences and the labels
# Separate the sentences and the labels for training and testing
train_x = list(corpus[corpus.split=='train'].sentence)
train_y = np.array(corpus[corpus.split=='train'].label)
print(len(train_x))
print(len(train_y))

test_x = list(corpus[corpus.split=='test'].sentence)
test_y = np.array(corpus[corpus.split=='test'].label)
print(len(test_x))
print(len(test_y))

5452
5452
500
500


In [7]:
train_x[0]

'how did serfdom develop in and then leave russia ?'

<!--## Split Dataset-->

# Data Preprocessing: Word2Vec Static
<hr>

Preparing data for word embedding, especially for pre-trained word embedding like Word2Vec or GloVe, __don't use standard preprocessing steps like stemming or stopword removal__. Compared to our approach on cleaning the text when doing word count based feature extraction (e.g. TFIDF) such as removing stopwords, stemming etc, now we will keep these words as we do not want to lose such information that might help the model learn better.

__Tomas Mikolov__, one of the developers of Word2Vec, in _word2vec-toolkit: google groups thread., 2015_, suggests only very minimal text cleaning is required when learning a word embedding model. Sometimes, it's good to disconnect
In short, what we will do is:
- Puntuations removal
- Lower the letter case
- Tokenization

The process above will be handled by __Tokenizer__ class in TensorFlow

## Load Pre-trained Word Embedding: Word2Vec

__1. Load `Word2Vec` Pre-trained Word Embedding__

__Using and updating pre-trained embeddings__
* In this part, we will create an Embedding layer in Tensorflow Keras using a pre-trained word embedding called Word2Vec 300-d tht has been trained 100 bilion words from Google News.
* In this part,  we will leave the embeddings fixed instead of updating them (dynamic).

In [8]:
from gensim.models import KeyedVectors
word2vec = KeyedVectors.load_word2vec_format('../GoogleNews-vectors-negative300.bin', binary=True)

In [9]:
# Access the dense vector value for the word 'handsome'
# word2vec.word_vec('handsome') # 0.11376953
word2vec.word_vec('cool') # 1.64062500e-01

array([ 1.64062500e-01,  1.87500000e-01, -4.10156250e-02,  1.25000000e-01,
       -3.22265625e-02,  8.69140625e-02,  1.19140625e-01, -1.26953125e-01,
        1.77001953e-02,  8.83789062e-02,  2.12402344e-02, -2.00195312e-01,
        4.83398438e-02, -1.01074219e-01, -1.89453125e-01,  2.30712891e-02,
        1.17675781e-01,  7.51953125e-02, -8.39843750e-02, -1.33666992e-02,
        1.53320312e-01,  4.08203125e-01,  3.80859375e-02,  3.36914062e-02,
       -4.02832031e-02, -6.88476562e-02,  9.03320312e-02,  2.12890625e-01,
        1.72119141e-02, -6.44531250e-02, -1.29882812e-01,  1.40625000e-01,
        2.38281250e-01,  1.37695312e-01, -1.76757812e-01, -2.71484375e-01,
       -1.36718750e-01, -1.69921875e-01, -9.15527344e-03,  3.47656250e-01,
        2.22656250e-01, -3.06640625e-01,  1.98242188e-01,  1.33789062e-01,
       -4.34570312e-02, -5.12695312e-02, -3.46679688e-02, -8.49609375e-02,
        1.01562500e-01,  1.42578125e-01, -7.95898438e-02,  1.78710938e-01,
        2.30468750e-01,  

In [11]:
def training_words_in_word2vector(word_to_vec_map, word_to_index):
    '''
    input:
        word_to_vec_map: a word2vec GoogleNews-vectors-negative300.bin model loaded using gensim.models
        word_to_index: word to index mapping from training set
    '''
    
    vocab_size = len(word_to_index) + 1
    count = 0
    # Set each row "idx" of the embedding matrix to be 
    # the word vector representation of the idx'th word of the vocabulary
    for word, idx in word_to_index.items():
        if word in word_to_vec_map:
            count+=1
            
    return print('Found {} words present from {} training vocabulary in the set of pre-trained word vector'.format(count, vocab_size))

In [12]:
oov_tok = '<UNK>'

# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

# Cleaning and Tokenization
tokenizer = Tokenizer(oov_token=oov_tok)
tokenizer.fit_on_texts(sentences)

word_index = tokenizer.word_index
training_words_in_word2vector(word2vec, word_index)

Found 7526 words present from 8761 training vocabulary in the set of pre-trained word vector


In [16]:
len(word_index)

8760

## Define `clean_doc` function
__2. Define a function to clean the document called __`clean_doc()`____

In [13]:
def clean_doc(sentences, word_index):
    clean_sentences = []
    for sentence in sentences:
        sentence = sentence.lower().split()
        clean_word = []
        for word in sentence:
            if word in word_index:
                clean_word.append(word)
        clean_sentence = ' '.join(clean_word)
        clean_sentences.append(clean_sentence)
    return clean_sentences

In [14]:
clean_sentences = clean_doc(sentences, word_index)
clean_sentences[0:3]

['how did serfdom develop in and then leave russia',
 'what films featured the character popeye doyle',
 "how can i find a list of celebrities ' real names"]

## Define `sentence_to_avg` function
__3. Define a `sentence_to_avg` function__

We will use this function to calculate the mean of word embedding representation.

In [17]:
def sentence_to_avg(sentence, word_to_vec_map):
    """
    Converts a sentence (string) into a list of words (strings). Extracts the GloVe representation of each word
    and averages its value into a single vector encoding the meaning of the sentence.
    
    Arguments:
    sentence -- string, one training example from X
    word_to_vec_map -- dictionary mapping every word in a vocabulary into its 50-dimensional vector representation
    
    Returns:
    avg -- average vector encoding information about the sentence, numpy-array of shape (50,)
    """
    
    # Step 1: Split sentence into list of lower case words (≈ 1 line)
    words = (sentence.lower()).split()

    # Initialize the average word vector, should have the same shape as your word vectors.
    avg = np.zeros(word2vec.word_vec('i').shape)
    
    # Step 2: average the word vectors. You can loop over the words in the list "words".
    total = 0
    count = 0
    for w in words:
        if w in word_to_vec_map:
            total += word_to_vec_map.word_vec(w)
            count += 1
            
    if count!= 0:
        avg = total/count
    else:
        avg
    return avg

In [18]:
i = word2vec.word_vec('i')[0]
print(word2vec.word_vec('i')[0])
j = word2vec.word_vec('am')[0]
print(word2vec.word_vec('am')[0])
k = word2vec.word_vec('handsome')[0]
print(word2vec.word_vec('handsome')[0])
mean = (i+j+k)/3
print('the mean of word embedding is: ', mean)

-0.22558594
-0.16699219
0.11376953
the mean of word embedding is:  -0.09293619791666667


In [19]:
# Example of the functions used in a sentence
mysentence = 'I am handsome'
sentence_to_avg(mysentence, word2vec)

array([-0.0929362 ,  0.03125   , -0.03914388,  0.09879557,  0.07088598,
        0.03092448, -0.00651042, -0.04437256,  0.08068848,  0.07242838,
        0.00160726, -0.10530599, -0.07389323, -0.08854166,  0.00565592,
        0.15136719, -0.0460612 ,  0.19482422,  0.1101888 ,  0.05924479,
       -0.18457031,  0.00716146,  0.16153972,  0.02437337, -0.01578776,
        0.06119792, -0.25048828,  0.02799479,  0.0853475 , -0.14029948,
        0.13688152, -0.01350911, -0.05493164, -0.01090495,  0.03352864,
        0.09635416, -0.04239909,  0.00777181, -0.1438802 ,  0.06510416,
        0.14560954, -0.11295573,  0.25520834,  0.08833822,  0.14339192,
        0.037028  , -0.02832031, -0.00139872,  0.00309245, -0.17871094,
        0.06852213,  0.07910156,  0.09513346,  0.11425781, -0.00488281,
        0.11051432, -0.01139323, -0.08479818, -0.09277344, -0.03263346,
       -0.00374349,  0.07977295, -0.26416016, -0.05135091,  0.06111654,
       -0.06933594, -0.06486002,  0.18766277, -0.04826609,  0.03

## Encode Sentence into Word2Vec Representation

In [20]:
def encoded_sentences(sentences):

    encoded_sentences = []

    for sentence in sentences:

        encoded_sentence = sentence_to_avg(sentence, word2vec)
        encoded_sentences.append(encoded_sentence)

    encoded_sentences = np.array(encoded_sentences)
    return encoded_sentences

In [21]:
embedded_sentences = encoded_sentences(clean_sentences)
print(embedded_sentences.shape)
embedded_sentences

(5952, 300)


array([[ 0.06625366,  0.01227188,  0.07821655, ..., -0.01962137,
         0.04184723, -0.01075745],
       [ 0.02888997,  0.08051554, -0.00393168, ..., -0.03134155,
        -0.00695801, -0.00199382],
       [ 0.03599548,  0.08511353,  0.02099609, ...,  0.00486755,
        -0.06564331, -0.03824615],
       ...,
       [ 0.0140625 ,  0.04776611,  0.09384765, ...,  0.01049805,
         0.03033447, -0.0977539 ],
       [ 0.03998413, -0.11558838,  0.03993492, ...,  0.01328125,
         0.1019638 , -0.03188477],
       [-0.00763702, -0.00300598,  0.16271973, ...,  0.01513672,
         0.10473633,  0.06201172]], dtype=float32)

# Saving to Matlab Files

In [22]:
# Parameter Initialization
oov_tok = "<UNK>"

data = {}

train_x = list(corpus[corpus.split=='train'].sentence)
train_y = np.array(corpus[corpus.split=='train'].label)
print(len(train_x))
print(len(train_y))

test_x = list(corpus[corpus.split=='test'].sentence)
test_y = np.array(corpus[corpus.split=='test'].label)
print(len(test_x))
print(len(test_y))
    
# Define the word_index based on training set
tokenizer = Tokenizer(oov_token=oov_tok)
tokenizer.fit_on_texts(train_x)
word_index = tokenizer.word_index

# Clean the sentences
Xtrain = clean_doc(train_x, word_index)
Xtest = clean_doc(test_x, word_index)

# Encode the sentences into word embedding average representation
Xtrain = encoded_sentences(Xtrain)
Xtest = encoded_sentences(Xtest)

data['Xtrain'] = Xtrain
data['ytrain'] = train_y
data['Xtest'] = Xtest
data['ytest'] = test_y

filename = 'we_TREC.mat'
savemat(filename, data)
print('The data is saved successfully!')

5452
5452
500
500
The data is saved successfully!


In [23]:
data

{'Xtrain': array([[ 0.06625366,  0.01227188,  0.07821655, ..., -0.01962137,
          0.04184723, -0.01075745],
        [ 0.02888997,  0.08051554, -0.00393168, ..., -0.03134155,
         -0.00695801, -0.00199382],
        [ 0.03599548,  0.08511353,  0.02099609, ...,  0.00486755,
         -0.06564331, -0.03824615],
        ...,
        [-0.01763306,  0.00096436,  0.08427735, ...,  0.06772461,
          0.09284668, -0.05214844],
        [-0.0278066 ,  0.04503377,  0.07877604, ...,  0.04292806,
          0.07348633,  0.02534994],
        [-0.00012716, -0.05644735,  0.00537109, ...,  0.03894043,
          0.00477091, -0.02848307]], dtype=float32),
 'ytrain': array([0, 1, 0, ..., 5, 5, 1]),
 'Xtest': array([[ 0.01940409, -0.02378988,  0.06665039, ..., -0.0544637 ,
          0.08583578, -0.04425049],
        [ 0.0383728 , -0.05054932,  0.1296875 , ..., -0.12749024,
          0.00762939, -0.12333985],
        [ 0.05255127,  0.0473938 ,  0.08734131, ..., -0.03442383,
          0.14648438, -0.0

In [24]:
data['Xtrain'].shape

(5452, 300)

In [25]:
data['Xtest'].shape

(500, 300)